In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
%matplotlib inline

In [2]:
train_x = np.load("s1_train_x_preprocess.npy")
train_y = np.load("s1_train_y.npy")
test_x = np.load("s1_test_x_preprocess.npy")

In [143]:
train_x = np.load("s1_train_x_no_preprocess.npy")
train_y = np.load("s1_train_y.npy")
test_x = np.load("s1_test_x_no_preprocess.npy")

In [3]:
def sign2loc(z):
    x = int((z - 100 - 1) / 6)
    y = int(((z-100-1) % 6))
    temp = np.zeros(12)
    temp[x] = 1
    temp[y + 6] = 1
    return  x, y

# 这里将训练集的数据还原成单通道的
$res\_x.shape = (60, 1, 510, 20)$  
$res\_y.shape = (60,)$  
$res\_t.shape=(50, 1, 510, 20)$

In [4]:
res_x = []
res_y = []
k = 0
for i in train_x:
    for j in i:
        res_x.append(j.reshape(1, j.shape[0], j.shape[1]))
        res_y.append(train_y[k])
    k += 1
res_x = np.array(res_x)
res_y = np.array(res_y)

In [5]:
res_t = [] 
for i in test_x:
    for j in i:
        res_t.append(j.reshape(1, j.shape[0], j.shape[1]))
res_t = np.array(res_t)

In [6]:
print(res_x.shape)
print(res_y.shape)
print(res_t.shape)

(60, 1, 510, 20)
(60,)
(50, 1, 510, 20)


In [7]:
res_y

array([102, 102, 102, 102, 102, 104, 104, 104, 104, 104, 107, 107, 107,
       107, 107, 112, 112, 112, 112, 112, 115, 115, 115, 115, 115, 117,
       117, 117, 117, 117, 119, 119, 119, 119, 119, 122, 122, 122, 122,
       122, 126, 126, 126, 126, 126, 130, 130, 130, 130, 130, 133, 133,
       133, 133, 133, 135, 135, 135, 135, 135], dtype=int64)

# 这里将数字分为行坐标和列坐标，然后使用两个相同的卷积去学习行和列来确定一个字符

In [8]:
res_row = []
rew_column = []
for i in res_y:
    x, y = sign2loc(i)
    res_row.append(x)
    rew_column.append(y)
res_row =  np.array(res_row)
rew_column =  np.array(rew_column)

In [9]:
res_row

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(       #(1*28*28)
                in_channels=1,
                out_channels=200,
                kernel_size=(3, 1),
                stride=1, #步长
#                 padding=2,
            ),    #(16*28*28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),#( 3, 253, 10)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(       #(1*28*28)
                in_channels=200,
                out_channels=200,
                kernel_size=(2, 2),
                stride=1, #步长
#                 padding=2,
            ),    #(16*28*28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4),#( 3, 253, 10)
        )
        
        self.fc1 = nn.Linear(200*31*1,64)  #全连接
        self.sf =nn.Softmax(dim=1)

        self.out1 = nn.Linear(64,6)  #全连接
#         self.out2 = nn.Linear(64,6)  #全连接

    def forward(self, x):
        x = self.conv1(x)
#         print(x.shape)
        x = self.conv2(x)
#         print(x.shape)
        
        x = x.view(x.size(0),-1) #(batch,32*7*7)
#         print(x.shape)
        x = F.relu(self.fc1(x))
        out1 = self.out1(x)
#         print(out1)
#         out2 = self.out2(x)
#         print(out2)
        return self.sf(out1)
#         return torch.cat((out1, out2), 0)
#         return output

In [11]:
train_t = torch.tensor(res_x).float()
train_y_t = torch.tensor(res_row).long()
train_y_c = torch.tensor(rew_column).long()

In [12]:
EPOCH = 300
LR = 0.001
# BATCH_SIZE = 32

In [15]:
cnn_row = CNN()
cnn_column = CNN()
optimizer_row = torch.optim.Adam(cnn_row.parameters(), lr=LR)
optimizer_column = torch.optim.Adam(cnn_column.parameters(), lr=LR)
loss_func_row = nn.CrossEntropyLoss()
loss_func_column = nn.CrossEntropyLoss()
losses_row = []
losses_column = []

In [ ]:
for epoch in range(EPOCH):
    output  = cnn_row(train_t)
    loss = loss_func_row(output, train_y_t)
    
    optimizer_row.zero_grad()
    loss.backward()
    optimizer_row.step()
#     predicted = torch.max(output.data, 1)[1]
#     print(predicted)
#     correct += (predicted == train_y_t).sum()
    print('Epoch : {} | Loss: {:.6f}\t'.format(epoch, loss.item()))
    losses_row.append(loss.item())
#     print('Epoch: ',epoch,'| train loss: ',loss)

Epoch : 0 | Loss: 1.791854	
Epoch : 1 | Loss: 1.789711	
Epoch : 2 | Loss: 1.787189	
Epoch : 3 | Loss: 1.782314	
Epoch : 4 | Loss: 1.777966	
Epoch : 5 | Loss: 1.769551	
Epoch : 6 | Loss: 1.759288	
Epoch : 7 | Loss: 1.762005	
Epoch : 8 | Loss: 1.760542	
Epoch : 9 | Loss: 1.760472	
Epoch : 10 | Loss: 1.740459	
Epoch : 11 | Loss: 1.738508	
Epoch : 12 | Loss: 1.735861	
Epoch : 13 | Loss: 1.708371	
Epoch : 14 | Loss: 1.711722	
Epoch : 15 | Loss: 1.695708	
Epoch : 16 | Loss: 1.691780	
Epoch : 17 | Loss: 1.671000	
Epoch : 18 | Loss: 1.666864	
Epoch : 19 | Loss: 1.649058	
Epoch : 20 | Loss: 1.638497	
Epoch : 21 | Loss: 1.619803	
Epoch : 22 | Loss: 1.606469	
Epoch : 23 | Loss: 1.588728	
Epoch : 24 | Loss: 1.577267	


In [16]:
for epoch in range(EPOCH):
    output  = cnn_column(train_t)
    loss = loss_func_column(output, train_y_c)
    
    optimizer_column.zero_grad()
    loss.backward()
    optimizer_column.step()
#     predicted = torch.max(output.data, 1)[1]
#     print(predicted)
#     correct += (predicted == train_y_t).sum()
    print('Epoch : {} | Loss: {:.6f}\t'.format(epoch, loss.item()))
    losses_column.append(loss.item())
#     print('Epoch: ',epoch,'| train loss: ',loss)

torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 0 | Loss: 1.792253	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 1 | Loss: 1.786576	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 2 | Loss: 1.778179	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 3 | Loss: 1.768218	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 4 | Loss: 1.755263	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 5 | Loss: 1.734281	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 6 | Loss: 1.709979	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 7 | Loss: 1.688204	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 8 | Loss: 1.650331	
torch.Size

Epoch : 73 | Loss: 1.043623	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 74 | Loss: 1.043622	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 75 | Loss: 1.043622	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 76 | Loss: 1.043622	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 77 | Loss: 1.043621	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 78 | Loss: 1.043621	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 79 | Loss: 1.043620	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 80 | Loss: 1.043620	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 81 | Loss: 1.043620	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
E

torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 147 | Loss: 1.043610	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 148 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 149 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 150 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 151 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 152 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 153 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 154 | Loss: 1.043609	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 155 | Loss: 1.

torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 220 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 221 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 222 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 223 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 224 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 225 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 226 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 227 | Loss: 1.043603	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 228 | Loss: 1.

torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 293 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 294 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 295 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 296 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 297 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 298 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 299 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 300 | Loss: 1.043593	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 301 | Loss: 1.

torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 366 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 367 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 368 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 369 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 370 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 371 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 372 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 373 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 374 | Loss: 1.

torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 439 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 440 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 441 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 442 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 443 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 444 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 445 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 446 | Loss: 1.043592	
torch.Size([12, 200, 127, 5])
torch.Size([12, 200, 31, 1])
torch.Size([12, 6200])
Epoch : 447 | Loss: 1.

In [17]:
tt = cnn_row(torch.tensor(test_x).float())
t1 = cnn_column(torch.tensor(test_x).float())
# torch.max(tt.data, 1)[1]

torch.Size([10, 200, 127, 5])
torch.Size([10, 200, 31, 1])
torch.Size([10, 6200])
torch.Size([10, 200, 127, 5])
torch.Size([10, 200, 31, 1])
torch.Size([10, 6200])


In [18]:
torch.max(tt.data, 1)[1]

tensor([2, 0, 5, 2, 0, 3, 3, 0, 2, 2])

In [19]:
torch.max(t1.data, 1)[1]

tensor([4, 4, 2, 4, 5, 3, 1, 0, 4, 4])

In [41]:
res_t.shape

(50, 1, 510, 20)